In [ ]:
from pgmpy.models import MarkovNetwork
import networkx as nx
import matplotlib.pyplot as plt  
import numpy as np
from pgmpy.factors.discrete import DiscreteFactor
from pandas import read_csv
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from tqdm import tqdm
from pgmpy.inference import VariableElimination
import logging.config

logging.config.dictConfig({
    'version': 1,
    'disable_existing_loggers': True,
})

seed = 1
tf.random.set_seed(seed)
np.random.seed(seed)

train_acc = []
train_mae = []
test_acc = []
test_mae = []

for i in range(1):
    print(i)
    G = MarkovNetwork()

    # number of inputs
    x_count = 4
    # number of nodes in the hidden layers
    h1_count = 6
    h2_count = 6

    weight_range = 10
    dnn_models = 1
    datapoints = 1000

    weights = {}

    G2 = nx.Graph()

    def potentials(edge, weights):
        return np.array([[np.exp(weights[(edge[0], edge[1])]),1],[1, 1]])

    for x in range(x_count):
        G.add_node('x%s'%x)
        G2.add_node('x%s'%x, subset=1)
    for h1 in range(h1_count):
        G.add_node('h1%s'%h1)
        G2.add_node('h1%s'%h1, subset=2)
        for x in range(x_count):
            G.add_edge('x%s'%x, 'h1%s'%h1)
            G2.add_edge('x%s'%x, 'h1%s'%h1)
            weights[('x%s'%x, 'h1%s'%h1)] = np.random.uniform(-weight_range,weight_range)
    h1 = 0
    for h2 in range(h2_count):
        G.add_node('h2%s'%h2)
        G2.add_node('h2%s'%h2, subset=3)
        G.add_edge('h1%s'%h1, 'h2%s'%h2)
        G2.add_edge('h1%s'%h1, 'h2%s'%h2)
        weights[('h1%s'%h1, 'h2%s'%h2)] = np.random.uniform(-weight_range,weight_range)
        h1 += 1
    # add the output node y
    G.add_node('y')
    G2.add_node('y', subset=4)
    for h2 in range(h2_count):
        G.add_edge('h2%s'%h2, 'y')
        G2.add_edge('h2%s'%h2, 'y')
        weights[('h2%s'%h2, 'y')] = np.random.uniform(-weight_range,weight_range)

    phi = [DiscreteFactor(edge, [2,2], potentials(edge, weights), state_names={str(edge[0]) : [1,0],
                                                                            str(edge[1]) : [1,0]}) for edge in G.edges()]
    G.add_factors(*phi)
    #nx.draw_circular(G, with_labels=True, arrows=True)
    #plt.show()

    pos=nx.multipartite_layout(G2)
    nx.draw(G2,pos)
    labels = nx.get_edge_attributes(G2,'weight')
    rounded_weights = weights
    for edge in rounded_weights: 
        rounded_weights[edge] = round(rounded_weights[edge],2)
    print(weights)
    nx.draw_networkx_edge_labels(G2,pos,edge_labels=rounded_weights)
    nx.draw_networkx_labels(G2,pos)
    plt.show()

    my_data = np.empty(shape=(datapoints,6))
    assignments = {}

    for i in tqdm(range(0,datapoints)):
        inputs = {'x0' : np.random.binomial(n = 1, p = 0.5),
                'x1' : np.random.binomial(n = 1, p = 0.33),
                'x2' : np.random.binomial(n = 1, p = 0.78),
                'x3' : np.random.binomial(n = 1, p = 0.45)}
        assignments.update(inputs)

        inference = VariableElimination(G)
        exact = inference.query(['y'], evidence=assignments).values
        y = np.random.binomial(n = 1, p = exact[0]/(exact[1]+exact[0]))

        my_data[i,0] = assignments['x0']
        my_data[i,1] = assignments['x1']
        my_data[i,2] = assignments['x2']
        my_data[i,3] = assignments['x3']
        my_data[i,4] = y
        my_data[i,5] = exact[0]/(exact[1]+exact[0])

    np.savetxt("mn_simulated.csv", my_data, delimiter=",")

    df = read_csv("mn_simulated.csv", header=None)

    X, y = df.values[:, :-2], df.values[:, -2:]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

    train_probs = y_train[:,1]
    y_train = y_train[:,0]

    test_probs = y_test[:,1]
    y_test = y_test[:,0]

    model_train_acc = []
    model_train_mae = []
    model_test_acc = []
    model_test_mae = []

    for j in range(dnn_models):
        # the model
        model = Sequential()
        model.add(Dense(32, activation='sigmoid', kernel_initializer='he_normal', input_shape=(4,)))
        model.add(Dense(32, activation='sigmoid', kernel_initializer='he_normal'))
        model.add(Dense(1, activation='sigmoid'))

        # compile the model
        opt = SGD(learning_rate=0.01)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'mean_absolute_error'])

        model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

        loss, acc, mae = model.evaluate(X_train, y_train, verbose=0)
        model_train_acc.append(acc)

        loss, acc, mae = model.evaluate(X_train, train_probs, verbose=0)
        model_train_mae.append(mae)

        loss, acc, mae = model.evaluate(X_test, y_test, verbose=0)
        model_test_acc.append(acc)

        loss, acc, mae = model.evaluate(X_test, test_probs, verbose=0)
        model_test_mae.append(mae)

    print('Train Accuracy: %s (%s)'%(np.mean(model_train_acc), np.std(model_train_acc)))
    print('Train MAE: %s (%s)'%(np.mean(model_train_mae), np.std(model_train_mae)))
    print('Test Accuracy: %s (%s)'%(np.mean(model_test_acc), np.std(model_test_acc)))
    print('Test MAE: %s (%s)'%(np.mean(model_test_mae), np.std(model_test_mae)))